In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 17.3 MB/s eta 0:00:00


case0

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")


Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case0.txt to Case0.txt
Fichier téléversé avec succès.
Nombre de clients : 3
Capacité des véhicules : 10
Demandes des clients : [4, 5, 6]
Coordonnées des nœuds : [(0, 0), (2, 3), (4, 5), (6, 7)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 4)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 4)), lowBound=0)

# Variables auxiliaires pour l'élimination des sous-tours
u = pulp.LpVariable.dicts("u", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 4)), lowBound=0, cat='Continuous')

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 4))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 4)) == demands[i - 1]

for k in range(1, max_vehicles + 4):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 4):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 4):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 4):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Ajout de la contrainte d'élimination des sous-tours
for k in range(1, max_vehicles + 4):
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if i != j:
                model += u[i, k] - u[j, k] + n * x[i, j, k] <= n - 1, f"EliminationSousTours_{i}_{j}_{k}"

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=1800, threads=4))

# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 4):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")


Routes:
Route 1: 0 -> 1 (4.0) -> 2 (5.0) -> 0
Route 2: 0 -> 3 (6.0) -> 0
Total cost (recalculated): 31
Number of deliveries: 3
Truck loads: 9.0 6.0


cas0

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 4)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 4)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 4))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 4)) == demands[i - 1]

for k in range(1, max_vehicles + 4):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 4):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 4):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 4):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)


# Résoudre avec une limite de temps
# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=1800, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 4):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")



Routes:
Route 1: 0 -> 3 (6.0) -> 0
Route 2: 0 -> 1 (4.0) -> 2 (5.0) -> 0
Total cost (recalculated): 31
Number of deliveries: 3
Truck loads: 6.0 9.0


Cas1

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")


Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case1.txt to Case1.txt
Fichier téléversé avec succès.
Nombre de clients : 8
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (0, 1000), (-1000, 0), (0, -1000), (2000, 0), (0, 2000), (-2000, 0), (0, -2000), ()]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=120))

# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")



Routes:
Route 1: 0 -> 2 (80.0) -> 3 (20.0) -> 0
Route 2: 0 -> 2 (10.0) -> 6 (90.0) -> 0
Route 3: 0 -> 4 (10.0) -> 8 (90.0) -> 0
Route 4: 0 -> 3 (40.0) -> 7 (60.0) -> 0
Route 5: 0 -> 1 (20.0) -> 4 (80.0) -> 0
Route 6: 0 -> 1 (40.0) -> 5 (60.0) -> 0
Total cost (recalculated): 22828
Number of deliveries: 12
Truck loads: 100.0 100.0 100.0 100.0 100.0 100.0


Case2

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case2.txt to Case2.txt
Fichier téléversé avec succès.
Nombre de clients : 16
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (0, 1000), (-1000, 0), (0, -1000), (2000, 0), (0, 2000), (-2000, 0), (0, -2000), (3000, 0), (0, 3000), (-3000, 0), (0, -3000), (4000, 0), (0, 4000), (-4000, 0), (-1, -4000), (), ()]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

12
Routes:
Route 1: 0 -> 10 (70.0) -> 14 (30.0) -> 0
Route 2: 0 -> 7 (20.0) -> 11 (20.0) -> 12 (60.0) -> 0
Route 3: 0 -> 6 (80.0) -> 10 (20.0) -> 0
Route 4: 0 -> 11 (40.0) -> 15 (60.0) -> 0
Route 5: 0 -> 8 (80.0) -> 12 (20.0) -> 0
Route 6: 0 -> 4 (90.0) -> 8 (10.0) -> 0
Route 7: 0 -> 5 (20.0) -> 9 (20.0) -> 14 (60.0) -> 0
Route 8: 0 -> 1 (60.0) -> 5 (40.0) -> 0
Route 9: 0 -> 12 (10.0) -> 16 (90.0) -> 0
Route 10: 0 -> 2 (90.0) -> 6 (10.0) -> 0
Route 11: 0 -> 9 (40.0) -> 13 (60.0) -> 0
Route 12: 0 -> 3 (60.0) -> 7 (40.0) -> 0
Total cost (recalculated): 82243
Number of deliveries: 26
Truck loads: 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0


Cas3

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case3.txt to Case3 (1).txt
Fichier téléversé avec succès.
Nombre de clients : 16
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (707, 707), (0, 1000), (-707, 707), (-1000, 0), (-707, -707), (0, -1000), (707, -707), (2000, 0), (1414, 1414), (0, 2000), (-1414, 1414), (-2000, 0), (-1414, -1414), (0, -2000), (1414, -1414), ()]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=18, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

12
Routes:
Route 1: 0 -> 2 (80.0) -> 3 (20.0) -> 0
Route 2: 0 -> 1 (40.0) -> 9 (60.0) -> 0
Route 3: 0 -> 6 (80.0) -> 7 (20.0) -> 0
Route 4: 0 -> 4 (80.0) -> 5 (20.0) -> 0
Route 5: 0 -> 6 (10.0) -> 14 (90.0) -> 0
Route 6: 0 -> 7 (40.0) -> 15 (60.0) -> 0
Route 7: 0 -> 5 (40.0) -> 13 (60.0) -> 0
Route 8: 0 -> 4 (10.0) -> 12 (90.0) -> 0
Route 9: 0 -> 2 (10.0) -> 10 (90.0) -> 0
Route 10: 0 -> 3 (40.0) -> 11 (60.0) -> 0
Route 11: 0 -> 1 (20.0) -> 8 (80.0) -> 0
Route 12: 0 -> 8 (10.0) -> 16 (90.0) -> 0
Total cost (recalculated): 43060
Number of deliveries: 24
Truck loads: 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0


Cas4

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case4.txt to Case4.txt
Fichier téléversé avec succès.
Nombre de clients : 24
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (866, 500), (500, 866), (0, 1000), (-500, 866), (-866, 500), (-1000, 0), (-866, -500), (-500, -866), (0, -1000), (500, -866), (866, -500), (2000, 0), (1732, 1000), (1000, 1732), (0, 2000), (-1000, 1732), (-1732, 1000), (-2000, 0), (-1732, -1000), (-1000, -1732), (0, -2000), (1000, -1732), (1732, -1000), (), ()]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

18
Routes:
Route 1: 0 -> 13 (60.0) -> 24 (40.0) -> 0
Route 2: 0 -> 11 (10.0) -> 12 (90.0) -> 0
Route 3: 0 -> 15 (50.0) -> 16 (50.0) -> 0
Route 4: 0 -> 18 (90.0) -> 19 (10.0) -> 0
Route 5: 0 -> 22 (90.0) -> 23 (10.0) -> 0
Route 6: 0 -> 14 (90.0) -> 15 (10.0) -> 0
Route 7: 0 -> 1 (60.0) -> 2 (40.0) -> 0
Route 8: 0 -> 2 (50.0) -> 3 (50.0) -> 0
Route 9: 0 -> 7 (60.0) -> 8 (40.0) -> 0
Route 10: 0 -> 16 (40.0) -> 17 (60.0) -> 0
Route 11: 0 -> 8 (50.0) -> 9 (50.0) -> 0
Route 12: 0 -> 19 (50.0) -> 20 (50.0) -> 0
Route 13: 0 -> 10 (50.0) -> 11 (50.0) -> 0
Route 14: 0 -> 3 (10.0) -> 4 (90.0) -> 0
Route 15: 0 -> 5 (50.0) -> 9 (10.0) -> 10 (40.0) -> 0
Route 16: 0 -> 20 (40.0) -> 21 (60.0) -> 0
Route 17: 0 -> 23 (50.0) -> 24 (50.0) -> 0
Route 18: 0 -> 5 (10.0) -> 6 (90.0) -> 0
Total cost (recalculated): 69709
Number of deliveries: 37
Truck loads: 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0


Cas5

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case5.txt to Case5.txt
Fichier téléversé avec succès.
Nombre de clients : 32
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (707, 707), (0, 1000), (-707, 707), (-1000, 0), (-707, -707), (0, -1000), (707, -707), (2000, 0), (1414, 1414), (0, 2000), (-1414, 1414), (-2000, 0), (-1414, -1414), (0, -2000), (1414, -1414), (3000, 0), (2121, 2121), (0, 3000), (-2121, 2121), (-3000, 0), (-2122, -2121), (0, -3000), (2121, -2122), (4000, 0), (2828, 2828), (0, 4000), (-2828, 2829), (-4000, 0), (-2829, -2828), (-1, -4000), (2828, -2829)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

24
Routes:
Route 1: 0 -> 13 (60.0) -> 21 (20.0) -> 30 (20.0) -> 0
Route 2: 0 -> 4 (10.0) -> 12 (90.0) -> 0
Route 3: 0 -> 18 (90.0) -> 26 (10.0) -> 0
Route 4: 0 -> 7 (20.0) -> 20 (80.0) -> 0
Route 5: 0 -> 5 (60.0) -> 6 (40.0) -> 0
Route 6: 0 -> 19 (40.0) -> 27 (60.0) -> 0
Route 7: 0 -> 8 (10.0) -> 16 (90.0) -> 0
Route 8: 0 -> 7 (40.0) -> 15 (60.0) -> 0
Route 9: 0 -> 2 (50.0) -> 10 (50.0) -> 0
Route 10: 0 -> 10 (40.0) -> 11 (40.0) -> 19 (20.0) -> 0
Route 11: 0 -> 14 (40.0) -> 22 (60.0) -> 0
Route 12: 0 -> 11 (20.0) -> 26 (60.0) -> 31 (20.0) -> 0
Route 13: 0 -> 26 (20.0) -> 32 (80.0) -> 0
Route 14: 0 -> 21 (40.0) -> 29 (60.0) -> 0
Route 15: 0 -> 23 (60.0) -> 31 (40.0) -> 0
Route 16: 0 -> 20 (10.0) -> 28 (90.0) -> 0
Route 17: 0 -> 17 (40.0) -> 25 (60.0) -> 0
Route 18: 0 -> 1 (40.0) -> 9 (60.0) -> 0
Route 19: 0 -> 6 (50.0) -> 14 (50.0) -> 0
Route 20: 0 -> 24 (90.0) -> 32 (10.0) -> 0
Route 21: 0 -> 22 (30.0) -> 30 (70.0) -> 0
Route 22: 0 -> 4 (80.0) -> 17 (20.0) -> 0
Route 23: 0 -> 2 (40.0) 

Cas6

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case6.txt to Case6.txt
Fichier téléversé avec succès.
Nombre de clients : 32
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (924, 383), (707, 707), (383, 924), (0, 1000), (-383, 924), (-707, 707), (-924, 383), (-1000, 0), (-924, -383), (-707, -707), (-383, -924), (0, -1000), (383, -924), (707, -707), (924, -383), (2000, 0), (1848, 765), (1414, 1414), (765, 1848), (0, 2000), (-765, 1848), (-1414, 1414), (-1848, 766), (-2000, 0), (-1848, -765), (-1414, -1414), (-766, -1848), (0, -2000), (765, -1848), (1414, -1414), (1848, -766)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)



# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

24
Routes:
Route 1: 0 -> 26 (50.0) -> 27 (50.0) -> 0
Route 2: 0 -> 13 (10.0) -> 14 (90.0) -> 0
Route 3: 0 -> 20 (50.0) -> 21 (50.0) -> 0
Route 4: 0 -> 27 (10.0) -> 28 (90.0) -> 0
Route 5: 0 -> 11 (10.0) -> 12 (90.0) -> 0
Route 6: 0 -> 4 (50.0) -> 13 (50.0) -> 0
Route 7: 0 -> 1 (60.0) -> 2 (40.0) -> 0
Route 8: 0 -> 19 (60.0) -> 20 (40.0) -> 0
Route 9: 0 -> 17 (50.0) -> 32 (50.0) -> 0
Route 10: 0 -> 3 (60.0) -> 4 (40.0) -> 0
Route 11: 0 -> 15 (10.0) -> 16 (90.0) -> 0
Route 12: 0 -> 2 (50.0) -> 29 (50.0) -> 0
Route 13: 0 -> 15 (50.0) -> 23 (50.0) -> 0
Route 14: 0 -> 25 (60.0) -> 26 (40.0) -> 0
Route 15: 0 -> 5 (10.0) -> 6 (90.0) -> 0
Route 16: 0 -> 17 (10.0) -> 18 (90.0) -> 0
Route 17: 0 -> 21 (10.0) -> 22 (90.0) -> 0
Route 18: 0 -> 29 (10.0) -> 30 (90.0) -> 0
Route 19: 0 -> 9 (60.0) -> 10 (40.0) -> 0
Route 20: 0 -> 23 (10.0) -> 24 (90.0) -> 0
Route 21: 0 -> 5 (50.0) -> 8 (50.0) -> 0
Route 22: 0 -> 31 (60.0) -> 32 (40.0) -> 0
Route 23: 0 -> 10 (50.0) -> 11 (50.0) -> 0
Route 24: 0 -> 7 (60

case7

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée (exemple : Case0.txt) :


Saving Case7.txt to Case7.txt
Fichier téléversé avec succès.
Nombre de clients : 40
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (0, 1000), (-1000, 0), (0, -1000), (2000, 0), (0, 2000), (-2000, 0), (0, -2000), (3000, 0), (0, 3000), (-3000, 0), (0, -3000), (4000, 0), (0, 4000), (-4000, 0), (-1, -4000), (5000, 0), (0, 5000), (-5000, 0), (-1, -5000), (6000, 0), (0, 6000), (-6000, 1), (-1, -6000), (7000, 0), (0, 7000), (-7000, 1), (-1, -7000), (8000, 0), (0, 8000), (-8000, 1), (-1, -8000), (9000, 0), (0, 9000), (-9000, 1), (-1, -9000), (10000, 0), (0, 10000), (-10000, 1), (-1, -10000)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

30
Routes:
Route 1: 0 -> 24 (80.0) -> 30 (20.0) -> 0
Route 2: 0 -> 11 (20.0) -> 14 (80.0) -> 0
Route 3: 0 -> 17 (20.0) -> 28 (80.0) -> 0
Route 4: 0 -> 11 (40.0) -> 15 (60.0) -> 0
Route 5: 0 -> 3 (60.0) -> 5 (20.0) -> 9 (20.0) -> 0
Route 6: 0 -> 2 (80.0) -> 35 (20.0) -> 0
Route 7: 0 -> 25 (60.0) -> 29 (20.0) -> 33 (20.0) -> 0
Route 8: 0 -> 2 (10.0) -> 6 (70.0) -> 30 (20.0) -> 0
Route 9: 0 -> 26 (70.0) -> 30 (30.0) -> 0
Route 10: 0 -> 19 (60.0) -> 23 (40.0) -> 0
Route 11: 0 -> 27 (40.0) -> 31 (60.0) -> 0
Route 12: 0 -> 33 (40.0) -> 37 (60.0) -> 0
Route 13: 0 -> 35 (40.0) -> 39 (60.0) -> 0
Route 14: 0 -> 12 (80.0) -> 26 (20.0) -> 0
Route 15: 0 -> 12 (10.0) -> 16 (90.0) -> 0
Route 16: 0 -> 22 (80.0) -> 30 (20.0) -> 0
Route 17: 0 -> 4 (80.0) -> 36 (20.0) -> 0
Route 18: 0 -> 9 (40.0) -> 13 (60.0) -> 0
Route 19: 0 -> 29 (40.0) -> 36 (60.0) -> 0
Route 20: 0 -> 10 (90.0) -> 14 (10.0) -> 0
Route 21: 0 -> 23 (20.0) -> 34 (80.0) -> 0
Route 22: 0 -> 1 (60.0) -> 5 (40.0) -> 0
Route 23: 0 -> 17 (40.0

Case8

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case8.txt to Case8.txt
Fichier téléversé avec succès.
Nombre de clients : 48
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (0, 1000), (-1000, 0), (0, -1000), (2000, 0), (0, 2000), (-2000, 0), (0, -2000), (3000, 0), (0, 3000), (-3000, 0), (0, -3000), (4000, 0), (0, 4000), (-4000, 0), (-1, -4000), (5000, 0), (0, 5000), (-5000, 0), (-1, -5000), (6000, 0), (0, 6000), (-6000, 1), (-1, -6000), (7000, 0), (0, 7000), (-7000, 1), (-1, -7000), (8000, 0), (0, 8000), (-8000, 1), (-1, -8000), (9000, 0), (0, 9000), (-9000, 1), (-1, -9000), (10000, 0), (0, 10000), (-10000, 1), (-1, -10000), (11000, 0), (1, 11000), (-11000, 1), (-2, -11000), (12000, 0), (1, 12000), (-12000, 1), (-2, -12000)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

36
Routes:
Route 1: 0 -> 10 (10.0) -> 14 (90.0) -> 0
Route 2: 0 -> 4 (80.0) -> 16 (20.0) -> 0
Route 3: 0 -> 26 (80.0) -> 43 (20.0) -> 0
Route 4: 0 -> 9 (20.0) -> 34 (70.0) -> 38 (10.0) -> 0
Route 5: 0 -> 26 (10.0) -> 30 (90.0) -> 0
Route 6: 0 -> 18 (90.0) -> 22 (10.0) -> 0
Route 7: 0 -> 2 (60.0) -> 29 (20.0) -> 33 (20.0) -> 0
Route 8: 0 -> 19 (60.0) -> 21 (20.0) -> 23 (20.0) -> 0
Route 9: 0 -> 43 (40.0) -> 47 (60.0) -> 0
Route 10: 0 -> 23 (20.0) -> 48 (80.0) -> 0
Route 11: 0 -> 17 (60.0) -> 21 (40.0) -> 0
Route 12: 0 -> 12 (30.0) -> 16 (70.0) -> 0
Route 13: 0 -> 42 (80.0) -> 45 (20.0) -> 0
Route 14: 0 -> 33 (40.0) -> 37 (60.0) -> 0
Route 15: 0 -> 41 (60.0) -> 45 (40.0) -> 0
Route 16: 0 -> 12 (60.0) -> 28 (40.0) -> 0
Route 17: 0 -> 9 (40.0) -> 13 (60.0) -> 0
Route 18: 0 -> 1 (40.0) -> 5 (60.0) -> 0
Route 19: 0 -> 3 (20.0) -> 40 (80.0) -> 0
Route 20: 0 -> 28 (10.0) -> 32 (90.0) -> 0
Route 21: 0 -> 35 (60.0) -> 39 (40.0) -> 0
Route 22: 0 -> 11 (60.0) -> 15 (40.0) -> 0
Route 23: 0 -> 23 (2

Case9

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case9.txt to Case9.txt
Fichier téléversé avec succès.
Nombre de clients : 48
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (866, 500), (500, 866), (0, 1000), (-500, 866), (-866, 500), (-1000, 0), (-866, -500), (-500, -866), (0, -1000), (500, -866), (866, -500), (2000, 0), (1732, 1000), (1000, 1732), (0, 2000), (-1000, 1732), (-1732, 1000), (-2000, 0), (-1732, -1000), (-1000, -1732), (0, -2000), (1000, -1732), (1732, -1000), (3000, 0), (2598, 1500), (1500, 2598), (0, 3000), (-1500, 2598), (-2598, 1500), (-3000, 0), (-2598, -1500), (-1500, -2598), (0, -3000), (1500, -2598), (2598, -1500), (4000, 0), (3464, 2000), (2000, 3464), (0, 4000), (-2000, 3464), (-3464, 2000), (-4000, 0), (-3464, -2000), (-2000, -3464), (-1, -4000), (1999, -3464), (346

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

36
Routes:
Route 1: 0 -> 14 (30.0) -> 15 (50.0) -> 30 (20.0) -> 0
Route 2: 0 -> 2 (40.0) -> 3 (60.0) -> 0
Route 3: 0 -> 31 (40.0) -> 43 (60.0) -> 0
Route 4: 0 -> 22 (50.0) -> 23 (50.0) -> 0
Route 5: 0 -> 35 (60.0) -> 47 (40.0) -> 0
Route 6: 0 -> 20 (80.0) -> 27 (20.0) -> 0
Route 7: 0 -> 28 (90.0) -> 40 (10.0) -> 0
Route 8: 0 -> 8 (40.0) -> 9 (60.0) -> 0
Route 9: 0 -> 32 (10.0) -> 44 (90.0) -> 0
Route 10: 0 -> 19 (10.0) -> 20 (10.0) -> 32 (80.0) -> 0
Route 11: 0 -> 4 (20.0) -> 40 (60.0) -> 47 (20.0) -> 0
Route 12: 0 -> 36 (80.0) -> 45 (20.0) -> 0
Route 13: 0 -> 15 (10.0) -> 16 (90.0) -> 0
Route 14: 0 -> 1 (60.0) -> 12 (40.0) -> 0
Route 15: 0 -> 4 (70.0) -> 5 (30.0) -> 0
Route 16: 0 -> 11 (50.0) -> 12 (50.0) -> 0
Route 17: 0 -> 36 (10.0) -> 48 (90.0) -> 0
Route 18: 0 -> 5 (30.0) -> 6 (50.0) -> 23 (10.0) -> 24 (10.0) -> 0
Route 19: 0 -> 18 (50.0) -> 19 (50.0) -> 0
Route 20: 0 -> 10 (90.0) -> 11 (10.0) -> 0
Route 21: 0 -> 27 (40.0) -> 39 (60.0) -> 0
Route 22: 0 -> 21 (60.0) -> 22 (40.0) ->

Case10

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case10.txt to Case10.txt
Fichier téléversé avec succès.
Nombre de clients : 64
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (924, 383), (707, 707), (383, 924), (0, 1000), (-383, 924), (-707, 707), (-924, 383), (-1000, 0), (-924, -383), (-707, -707), (-383, -924), (0, -1000), (383, -924), (707, -707), (924, -383), (2000, 0), (1848, 765), (1414, 1414), (765, 1848), (0, 2000), (-765, 1848), (-1414, 1414), (-1848, 766), (-2000, 0), (-1848, -765), (-1414, -1414), (-766, -1848), (0, -2000), (765, -1848), (1414, -1414), (1848, -766), (3000, 0), (2772, 1148), (2121, 2121), (1148, 2772), (0, 3000), (-1148, 2772), (-2121, 2121), (-2772, 1148), (-3000, 0), (-2772, -1148), (-2122, -2121),

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=60, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

48
No optimal solution found. Solver status: Not Solved


cas11

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case11.txt to Case11.txt
Fichier téléversé avec succès.
Nombre de clients : 80
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (0, 1000), (-1000, 0), (0, -1000), (2000, 0), (0, 2000), (-2000, 0), (0, -2000), (3000, 0), (0, 3000), (-3000, 0), (0, -3000), (4000, 0), (0, 4000), (-4000, 0), (-1, -4000), (5000, 0), (0, 5000), (-5000, 0), (-1, -5000), (6000, 0), (0, 6000), (-6000, 1), (-1, -6000), (7000, 0), (0, 7000), (-7000, 1), (-1, -7000), (8000, 0), (0, 8000), (-8000, 1), (-1, -8000), (9000, 0), (0, 9000), (-9000, 1), (-1, -9000), (10000, 0), (0, 10000), (-10000, 1), (-1, -10000), (11000, 0), (1, 11000), (-11000, 1), 

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 10))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=60, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

60
No optimal solution found. Solver status: Not Solved


CASE12

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case12.txt to Case12.txt
Fichier téléversé avec succès.
Nombre de clients : 80
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (707, 707), (0, 1000), (-707, 707), (-1000, 0), (-707, -707), (0, -1000), (707, -707), (2000, 0), (1414, 1414), (0, 2000), (-1414, 1414), (-2000, 0), (-1414, -1414), (0, -2000), (1414, -1414), (3000, 0), (2121, 2121), (0, 3000), (-2121, 2121), (-3000, 0), (-2122, -2121), (0, -3000), (2121, -2122), (4000, 0), (2828, 2828), (0, 4000), (-2828, 2829), (-4000, 0), (-2829, -2828), (-1, -4000), (2828, -2829), (5000, 0), (3536, 3535), (0, 5000), (-3535, 3536), (-5000, 0), (-3536, -3535), (-1, -5000),

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

72
No optimal solution found. Solver status: Not Solved


Case13

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case13.txt to Case13.txt
Fichier téléversé avec succès.
Nombre de clients : 96
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (707, 707), (0, 1000), (-707, 707), (-1000, 0), (-707, -707), (0, -1000), (707, -707), (2000, 0), (1414, 1414), (0, 2000), (-1414, 1414), (-2000, 0), (-1414, -1414), (0, -2000), (1414, -1414), (3000, 0), (2121, 2121), (0, 3000), (-2121, 2121), (-3000, 0), (-2122, -2121), (0, -3000), (2121, -2122), (4000, 0), (2828, 2828), (0, 4000), (-2828, 2829), (-4000, 0), (-2829, -2828), (-1, -4000), (2828, -2829), (5000, 0), (3536, 3535), (0

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

72
No optimal solution found. Solver status: Not Solved


Case14

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case14.txt to Case14.txt
Fichier téléversé avec succès.
Nombre de clients : 120
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90]
Coordonnées des nœuds : [(0, 0), (1000, 0), (866, 500), (500, 866), (0, 1000), (-500, 866), (-866, 500), (-1000, 0), (-866, -500), (-500, -866), (0, -1000), (500, -866), (866, -500), (2000, 0), (1732, 1000), (1000, 1732), (0, 2000), (-1000, 1732), (-1732, 1000), (-2000, 0), (-1732, -1000), (-1000, -1732), (0, -2000), (1000, -1732), (1732, -1000), (3000, 0), (2598, 1500), (1500,

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

90
No optimal solution found. Solver status: Not Solved


cas15

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

Case16

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

Case17

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

Case18

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

Cas19

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

Cas20

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

Case21

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case21.txt to Case21.txt
Fichier téléversé avec succès.
Nombre de clients : 288
Capacité des véhicules : 100
Demandes des clients : [60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 90, 60, 

In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

216


cas22

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case22.txt to Case22.txt
Fichier téléversé avec succès.
Nombre de clients : 21
Capacité des véhicules : 6000
Demandes des clients : [1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2500, 1800, 700]
Coordonnées des nœuds : [(145, 215), (151, 264), (159, 261), (130, 254), (128, 252), (163, 247), (146, 246), (161, 242), (142, 239), (163, 236), (148, 232), (128, 231), (156, 217), (129, 214), (146, 208), (164, 208), (141, 206), (147, 193), (164, 193), (129, 189), (155, 185), (139, 182)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

4
Routes:
Route 1: 0 -> 3 (800.0) -> 4 (1400.0) -> 17 (1000.0) -> 18 (900.0) -> 20 (1800.0) -> 0
Route 2: 0 -> 5 (2100.0) -> 7 (600.0) -> 12 (1300.0) -> 15 (900.0) -> 0
Route 3: 0 -> 1 (1100.0) -> 2 (700.0) -> 6 (400.0) -> 7 (200.0) -> 8 (100.0) -> 9 (500.0) -> 10 (600.0) -> 14 (300.0) -> 16 (2100.0) -> 0
Route 4: 0 -> 11 (1200.0) -> 13 (1300.0) -> 19 (2500.0) -> 21 (700.0) -> 0
Total cost (recalculated): 567
Number of deliveries: 22
Truck loads: 5900.0 4900.0 6000.0 5700.0


Cas23

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case23.txt to Case23.txt
Fichier téléversé avec succès.
Nombre de clients : 22
Capacité des véhicules : 4500
Demandes des clients : [125, 84, 60, 500, 300, 175, 350, 150, 1100, 4100, 225, 300, 250, 500, 150, 100, 250, 120, 600, 500, 175, 75]
Coordonnées des nœuds : [(266, 235), (295, 272), (301, 258), (309, 260), (217, 274), (218, 278), (282, 267), (242, 249), (230, 262), (249, 268), (256, 267), (265, 257), (267, 242), (259, 265), (315, 233), (329, 252), (318, 252), (329, 224), (267, 213), (275, 192), (303, 201), (208, 217), (326, 181)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

3
Routes:
Route 1: 0 -> 9 (1100.0) -> 10 (1369.0) -> 0
Route 2: 0 -> 2 (84.0) -> 3 (60.0) -> 4 (500.0) -> 5 (300.0) -> 10 (2731.0) -> 13 (250.0) -> 20 (500.0) -> 22 (75.0) -> 0
Route 3: 0 -> 1 (125.0) -> 6 (175.0) -> 7 (350.0) -> 8 (150.0) -> 11 (225.0) -> 12 (300.0) -> 14 (500.0) -> 15 (150.0) -> 16 (100.0) -> 17 (250.0) -> 18 (120.0) -> 19 (600.0) -> 21 (175.0) -> 0
Total cost (recalculated): 963
Number of deliveries: 23
Truck loads: 2469.0 4500.0 3220.0


cas24

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case24.txt to Case24.txt
Fichier téléversé avec succès.
Nombre de clients : 29
Capacité des véhicules : 4500
Demandes des clients : [300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150, 1000]
Coordonnées des nœuds : [(162, 354), (218, 382), (218, 358), (201, 370), (214, 371), (224, 370), (210, 382), (104, 354), (126, 338), (119, 340), (129, 349), (126, 347), (125, 346), (116, 355), (126, 335), (125, 355), (119, 357), (115, 341), (153, 351), (175, 363), (180, 360), (159, 331), (188, 357), (152, 349), (215, 389), (212, 394), (188, 393), (207, 406), (184, 410), (207, 392)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

3
Routes:
Route 1: 0 -> 7 (150.0) -> 8 (450.0) -> 9 (300.0) -> 10 (100.0) -> 11 (550.0) -> 13 (150.0) -> 14 (150.0) -> 17 (100.0) -> 18 (150.0) -> 19 (400.0) -> 20 (300.0) -> 22 (100.0) -> 27 (100.0) -> 29 (1000.0) -> 0
Route 2: 0 -> 15 (550.0) -> 16 (150.0) -> 21 (1500.0) -> 23 (300.0) -> 24 (500.0) -> 25 (800.0) -> 26 (300.0) -> 28 (150.0) -> 0
Route 3: 0 -> 1 (300.0) -> 2 (3100.0) -> 3 (125.0) -> 4 (100.0) -> 5 (200.0) -> 6 (150.0) -> 11 (400.0) -> 12 (125.0) -> 0
Total cost (recalculated): 933
Number of deliveries: 30
Truck loads: 4000.0 4250.0 4500.0


Cas25

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case25.txt to Case25.txt
Fichier téléversé avec succès.
Nombre de clients : 32
Capacité des véhicules : 8000
Demandes des clients : [700, 400, 400, 1200, 40, 80, 2000, 900, 600, 750, 1500, 150, 250, 1600, 450, 700, 550, 650, 200, 400, 300, 1300, 700, 750, 1400, 4000, 600, 1000, 500, 2500, 1700, 1100]
Coordonnées des nœuds : [(292, 495), (298, 427), (309, 445), (307, 464), (336, 475), (320, 439), (321, 437), (322, 437), (323, 433), (324, 433), (323, 429), (314, 435), (311, 442), (304, 427), (293, 421), (296, 418), (261, 384), (297, 410), (315, 407), (314, 406), (321, 391), (321, 398), (314, 394), (313, 378), (304, 382), (295, 402), (283, 406), (279, 399), (271, 401), (264, 414), (277, 439), (290, 434), (319, 433)]


In [ ]:
import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

4
Routes:
Route 1: 0 -> 3 (400.0) -> 4 (1200.0) -> 23 (700.0) -> 24 (750.0) -> 26 (4000.0) -> 27 (600.0) -> 0
Route 2: 0 -> 1 (700.0) -> 2 (400.0) -> 5 (40.0) -> 8 (900.0) -> 9 (600.0) -> 10 (750.0) -> 12 (150.0) -> 13 (250.0) -> 16 (700.0) -> 20 (400.0) -> 21 (300.0) -> 22 (1300.0) -> 28 (1000.0) -> 29 (500.0) -> 0
Route 3: 0 -> 17 (550.0) -> 25 (1400.0) -> 30 (2500.0) -> 31 (1700.0) -> 0
Route 4: 0 -> 6 (80.0) -> 7 (2000.0) -> 11 (1500.0) -> 14 (1600.0) -> 15 (450.0) -> 18 (650.0) -> 19 (200.0) -> 32 (1100.0) -> 0
Total cost (recalculated): 1172
Number of deliveries: 32
Truck loads: 7650.0 7990.0 6150.0 7580.0


Cas26

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case26.txt to Case26.txt
Fichier téléversé avec succès.
Nombre de clients : 50
Capacité des véhicules : 160
Demandes des clients : [7, 30, 16, 9, 21, 15, 19, 23, 11, 5, 19, 29, 23, 21, 10, 15, 3, 41, 9, 28, 8, 8, 16, 10, 28, 7, 15, 14, 6, 19, 11, 12, 23, 26, 17, 6, 9, 15, 14, 7, 27, 13, 11, 16, 10, 5, 25, 17, 18, 10]
Coordonnées des nœuds : [(30, 40), (37, 52), (49, 49), (52, 64), (20, 26), (40, 30), (21, 47), (17, 63), (31, 62), (52, 33), (51, 21), (42, 41), (31, 32), (5, 25), (12, 42), (36, 16), (52, 41), (27, 23), (17, 33), (13, 13), (57, 58), (62, 42), (42, 57), (16, 57), (8, 52), (7, 38), (27, 68), (30, 48), (43, 67), (58, 48), (58, 27), (37, 69), (38, 46), (46, 10), (61, 33), (62, 63), (63, 69), (32, 22), (45, 35), (59, 15), (5, 6), (10, 17), (21, 10), (5, 64), (30, 15), (39, 10), (32, 39), (25, 32), (25, 55), (48, 28), (56, 37)]


In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

5
Routes:
Route 1: 0 -> 3 (16.0) -> 6 (15.0) -> 10 (5.0) -> 20 (28.0) -> 27 (15.0) -> 28 (14.0) -> 31 (11.0) -> 35 (17.0) -> 36 (6.0) -> 39 (14.0) -> 48 (17.0) -> 0
Route 2: 0 -> 9 (11.0) -> 14 (21.0) -> 24 (10.0) -> 25 (28.0) -> 30 (19.0) -> 34 (26.0) -> 43 (11.0) -> 49 (18.0) -> 0
Route 3: 0 -> 1 (7.0) -> 8 (23.0) -> 11 (19.0) -> 13 (23.0) -> 15 (10.0) -> 22 (8.0) -> 26 (7.0) -> 32 (12.0) -> 41 (27.0) -> 44 (16.0) -> 46 (5.0) -> 0
Route 4: 0 -> 2 (30.0) -> 7 (19.0) -> 12 (29.0) -> 16 (15.0) -> 21 (8.0) -> 23 (16.0) -> 29 (6.0) -> 47 (25.0) -> 50 (10.0) -> 0
Route 5: 0 -> 4 (9.0) -> 5 (21.0) -> 17 (3.0) -> 18 (41.0) -> 19 (9.0) -> 33 (23.0) -> 37 (9.0) -> 38 (15.0) -> 40 (7.0) -> 42 (13.0) -> 45 (10.0) -> 0
Total cost (recalculated): 1517
Number of deliveries: 50
Truck loads: 158.0 144.0 157.0 158.0 160.0


Cas27

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case27.txt to Case27.txt
Fichier téléversé avec succès.
Nombre de clients : 75
Capacité des véhicules : 140
Demandes des clients : [18, 26, 11, 30, 21, 19, 15, 16, 29, 26, 37, 16, 12, 31, 8, 19, 20, 13, 15, 22, 28, 12, 6, 27, 14, 18, 17, 29, 13, 22, 25, 28, 27, 19, 10, 12, 14, 24, 16, 33, 15, 11, 18, 17, 21, 27, 19, 20, 5, 22, 12, 19, 22, 16, 7, 26, 14, 21, 24, 13, 15, 18, 11, 28, 9, 37, 30, 10, 8, 11, 3, 1, 6, 10, 20]
Coordonnées des nœuds : [(40, 40), (22, 22), (36, 26), (21, 45), (45, 35), (55, 20), (33, 34), (50, 50), (55, 45), (26, 59), (40, 66), (55, 65), (35, 51), (62, 35), (62, 57), (62, 24), (21, 36), (33, 44), (9, 56), (62, 48), (66, 14), (44, 13), (26, 13), (11, 28), (7, 43), (17, 64), (41, 46), (55, 34), (35, 16), (52, 26), (43, 26), (31, 76), (22, 53), (26, 29), (50, 40), (55, 50), (54, 10), (60, 15), (47, 66), (30, 60), (30, 50), (12, 17), (15, 14), (16, 19), (21, 48), (50, 30), (51, 42), (50, 15), (48, 21), (12, 38), (15, 56), (29, 39), (54, 38), (55, 57), (67, 41

In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

5
Routes:
Route 1: 0 -> 2 (30.0) -> 12 (29.0) -> 16 (15.0) -> 21 (8.0) -> 28 (14.0) -> 29 (6.0) -> 31 (11.0) -> 46 (5.0) -> 47 (25.0) -> 50 (10.0) -> 0
Route 2: 0 -> 9 (11.0) -> 10 (5.0) -> 24 (10.0) -> 30 (19.0) -> 33 (23.0) -> 34 (26.0) -> 39 (14.0) -> 43 (11.0) -> 45 (10.0) -> 49 (18.0) -> 0
Route 3: 0 -> 1 (7.0) -> 7 (19.0) -> 8 (23.0) -> 11 (19.0) -> 15 (10.0) -> 22 (8.0) -> 23 (16.0) -> 26 (7.0) -> 32 (12.0) -> 35 (17.0) -> 36 (6.0) -> 44 (16.0) -> 0
Route 4: 0 -> 3 (16.0) -> 4 (9.0) -> 5 (21.0) -> 18 (41.0) -> 19 (9.0) -> 20 (28.0) -> 38 (15.0) -> 40 (7.0) -> 42 (13.0) -> 0
Route 5: 0 -> 6 (15.0) -> 13 (23.0) -> 14 (21.0) -> 17 (3.0) -> 25 (28.0) -> 27 (15.0) -> 37 (9.0) -> 41 (27.0) -> 48 (17.0) -> 0
Total cost (recalculated): 1549
Number of deliveries: 50
Truck loads: 153.0 147.0 160.0 159.0 158.0


Cas28

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case28.txt to Case28.txt
Fichier téléversé avec succès.
Nombre de clients : 100
Capacité des véhicules : 200
Demandes des clients : [10, 7, 13, 19, 26, 3, 5, 9, 16, 16, 12, 19, 23, 20, 8, 19, 2, 12, 17, 9, 11, 18, 29, 3, 6, 17, 16, 16, 9, 21, 27, 23, 11, 14, 8, 5, 8, 16, 31, 9, 5, 5, 7, 18, 16, 1, 27, 36, 30, 13, 10, 9, 14, 18, 2, 6, 7, 18, 28, 3, 13, 19, 10, 9, 20, 25, 25, 36, 6, 5, 15, 25, 9, 8, 18, 13, 14, 3, 23, 6, 26, 16, 11, 7, 41, 35, 26, 9, 15, 3, 1, 2, 22, 27, 20, 11, 12, 10, 9, 17]
Coordonnées des nœuds : [(35, 35), (41, 49), (35, 17), (55, 45), (55, 20), (15, 30), (25, 30), (20, 50), (10, 43), (55, 60), (30, 60), (20, 65), (50, 35), (30, 25), (15, 10), (30, 5), (10, 20), (5, 30), (20, 40), (15, 60), (45, 65), (45, 20), (45, 10), (55, 5), (65, 35), (65, 20), (45, 30), (35, 40), (41, 37), (64, 42), (40, 60), (31, 52), (35, 69), (53, 52), (65, 55), (63, 65), (2, 60), (20, 20), (5, 5), (60, 12), (40, 25), (42, 7), (24, 12), (23, 3), (11, 14), (6, 38), (2, 48), (8, 56), (1

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 35.3 MB/s eta 0:00:00


In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

8
Routes:
Route 1: 0 -> 2 (7.0) -> 14 (20.0) -> 15 (8.0) -> 22 (18.0) -> 35 (8.0) -> 38 (16.0) -> 41 (5.0) -> 43 (7.0) -> 50 (13.0) -> 53 (9.0) -> 57 (7.0) -> 60 (3.0) -> 65 (20.0) -> 71 (15.0) -> 76 (13.0) -> 83 (11.0) -> 96 (11.0) -> 99 (9.0) -> 0
Route 2: 0 -> 4 (19.0) -> 13 (23.0) -> 27 (16.0) -> 28 (16.0) -> 34 (14.0) -> 54 (18.0) -> 78 (3.0) -> 85 (41.0) -> 93 (22.0) -> 94 (27.0) -> 0
Route 3: 0 -> 1 (10.0) -> 18 (12.0) -> 21 (11.0) -> 33 (11.0) -> 52 (9.0) -> 59 (28.0) -> 68 (36.0) -> 69 (6.0) -> 72 (25.0) -> 73 (9.0) -> 74 (8.0) -> 80 (6.0) -> 81 (26.0) -> 92 (2.0) -> 0
Route 4: 0 -> 3 (13.0) -> 6 (3.0) -> 20 (9.0) -> 24 (3.0) -> 29 (9.0) -> 36 (5.0) -> 37 (8.0) -> 39 (31.0) -> 47 (27.0) -> 67 (25.0) -> 77 (14.0) -> 79 (23.0) -> 89 (15.0) -> 98 (10.0) -> 0
Route 5: 0 -> 7 (5.0) -> 10 (16.0) -> 25 (6.0) -> 40 (9.0) -> 44 (18.0) -> 48 (36.0) -> 55 (2.0) -> 58 (18.0) -> 62 (19.0) -> 82 (16.0) -> 86 (35.0) -> 0
Route 6: 0 -> 8 (9.0) -> 16 (19.0) -> 32 (23.0) -> 46 (1.0) -> 61 (13.0

cas29

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case29.txt to Case29.txt
Fichier téléversé avec succès.
Nombre de clients : 75
Capacité des véhicules : 100
Demandes des clients : [18, 26, 11, 30, 21, 19, 15, 16, 29, 26, 37, 16, 12, 31, 8, 19, 20, 13, 15, 22, 28, 12, 6, 27, 14, 18, 17, 29, 13, 22, 25, 28, 27, 19, 10, 12, 14, 24, 16, 33, 15, 11, 18, 17, 21, 27, 19, 20, 5, 22, 12, 19, 22, 16, 7, 26, 14, 21, 24, 13, 15, 18, 11, 28, 9, 37, 30, 10, 8, 11, 3, 1, 6, 10, 20]
Coordonnées des nœuds : [(40, 40), (22, 22), (36, 26), (21, 45), (45, 35), (55, 20), (33, 34), (50, 50), (55, 45), (26, 59), (40, 66), (55, 65), (35, 51), (62, 35), (62, 57), (62, 24), (21, 36), (33, 44), (9, 56), (62, 48), (66, 14), (44, 13), (26, 13), (11, 28), (7, 43), (17, 64), (41, 46), (55, 34), (35, 16), (52, 26), (43, 26), (31, 76), (22, 53), (26, 29), (50, 40), (55, 50), (54, 10), (60, 15), (47, 66), (30, 60), (30, 50), (12, 17), (15, 14), (16, 19), (21, 48), (50, 30), (51, 42), (50, 15), (48, 21), (12, 38), (15, 56), (29, 39), (54, 38), (55, 57), (67, 41

In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

14
Routes:
Route 1: 0 -> 6 (19.0) -> 11 (4.0) -> 22 (12.0) -> 62 (18.0) -> 66 (37.0) -> 68 (10.0) -> 0
Route 2: 0 -> 27 (17.0) -> 41 (15.0) -> 43 (18.0) -> 52 (19.0) -> 60 (13.0) -> 71 (3.0) -> 0
Route 3: 0 -> 12 (16.0) -> 25 (14.0) -> 28 (6.0) -> 29 (13.0) -> 40 (33.0) -> 45 (6.0) -> 55 (7.0) -> 0
Route 4: 0 -> 11 (33.0) -> 17 (20.0) -> 21 (20.0) -> 51 (12.0) -> 61 (15.0) -> 0
Route 5: 0 -> 2 (26.0) -> 8 (16.0) -> 19 (15.0) -> 20 (10.0) -> 30 (22.0) -> 37 (11.0) -> 0
Route 6: 0 -> 3 (11.0) -> 18 (13.0) -> 26 (18.0) -> 44 (6.0) -> 50 (22.0) -> 67 (30.0) -> 0
Route 7: 0 -> 1 (18.0) -> 7 (15.0) -> 33 (27.0) -> 35 (10.0) -> 58 (21.0) -> 72 (1.0) -> 0
Route 8: 0 -> 4 (30.0) -> 20 (12.0) -> 21 (8.0) -> 47 (19.0) -> 70 (11.0) -> 75 (20.0) -> 0
Route 9: 0 -> 10 (26.0) -> 13 (12.0) -> 31 (25.0) -> 36 (12.0) -> 54 (16.0) -> 69 (8.0) -> 0
Route 10: 0 -> 16 (19.0) -> 38 (24.0) -> 45 (15.0) -> 53 (22.0) -> 63 (11.0) -> 65 (9.0) -> 0
Route 11: 0 -> 14 (31.0) -> 24 (27.0) -> 49 (5.0) -> 59 (24.0) ->

Cas 30

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case30.txt to Case30.txt
Fichier téléversé avec succès.
Nombre de clients : 100
Capacité des véhicules : 112
Demandes des clients : [10, 7, 13, 19, 26, 3, 5, 9, 16, 16, 12, 19, 23, 20, 8, 19, 2, 12, 17, 9, 11, 18, 29, 3, 6, 17, 16, 16, 9, 21, 27, 23, 11, 14, 8, 5, 8, 16, 31, 9, 5, 5, 7, 18, 16, 1, 27, 36, 30, 13, 10, 9, 14, 18, 2, 6, 7, 18, 28, 3, 13, 19, 10, 9, 20, 25, 25, 36, 6, 5, 15, 25, 9, 8, 18, 13, 14, 3, 23, 6, 26, 16, 11, 7, 41, 35, 26, 9, 15, 3, 1, 2, 22, 27, 20, 11, 12, 10, 9, 17]
Coordonnées des nœuds : [(35, 35), (41, 49), (35, 17), (55, 45), (55, 20), (15, 30), (25, 30), (20, 50), (10, 43), (55, 60), (30, 60), (20, 65), (50, 35), (30, 25), (15, 10), (30, 5), (10, 20), (5, 30), (20, 40), (15, 60), (45, 65), (45, 20), (45, 10), (55, 5), (65, 35), (65, 20), (45, 30), (35, 40), (41, 37), (64, 42), (40, 60), (31, 52), (35, 69), (53, 52), (65, 55), (63, 65), (2, 60), (20, 20), (5, 5), (60, 12), (40, 25), (42, 7), (24, 12), (23, 3), (11, 14), (6, 38), (2, 48), (8, 56), (1

In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

14
Routes:
Route 1: 0 -> 9 (16.0) -> 20 (9.0) -> 32 (13.0) -> 33 (11.0) -> 51 (10.0) -> 66 (25.0) -> 81 (26.0) -> 0
Route 2: 0 -> 2 (7.0) -> 12 (19.0) -> 26 (17.0) -> 31 (27.0) -> 50 (13.0) -> 57 (7.0) -> 76 (13.0) -> 88 (9.0) -> 0
Route 3: 0 -> 30 (21.0) -> 32 (10.0) -> 63 (10.0) -> 70 (5.0) -> 85 (41.0) -> 90 (3.0) -> 93 (22.0) -> 0
Route 4: 0 -> 5 (26.0) -> 8 (9.0) -> 17 (2.0) -> 23 (29.0) -> 45 (16.0) -> 46 (1.0) -> 75 (18.0) -> 84 (7.0) -> 0
Route 5: 0 -> 24 (3.0) -> 29 (9.0) -> 40 (9.0) -> 53 (14.0) -> 58 (18.0) -> 65 (20.0) -> 71 (15.0) -> 91 (1.0) -> 100 (17.0) -> 0
Route 6: 0 -> 4 (19.0) -> 13 (15.0) -> 14 (20.0) -> 44 (18.0) -> 54 (18.0) -> 0
Route 7: 0 -> 27 (16.0) -> 28 (16.0) -> 37 (8.0) -> 48 (13.0) -> 72 (25.0) -> 74 (8.0) -> 82 (16.0) -> 98 (10.0) -> 0
Route 8: 0 -> 3 (6.0) -> 7 (5.0) -> 11 (12.0) -> 19 (17.0) -> 25 (6.0) -> 48 (23.0) -> 55 (2.0) -> 79 (23.0) -> 0
Route 9: 0 -> 13 (8.0) -> 36 (5.0) -> 47 (27.0) -> 49 (30.0) -> 56 (6.0) -> 64 (9.0) -> 94 (27.0) -> 0
Rout

Cas31

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case31.txt to Case31.txt
Fichier téléversé avec succès.
Nombre de clients : 75
Capacité des véhicules : 180
Demandes des clients : [18, 26, 11, 30, 21, 19, 15, 16, 29, 26, 37, 16, 12, 31, 8, 19, 20, 13, 15, 22, 28, 12, 6, 27, 14, 18, 17, 29, 13, 22, 25, 28, 27, 19, 10, 12, 14, 24, 16, 33, 15, 11, 18, 17, 21, 27, 19, 20, 5, 22, 12, 19, 22, 16, 7, 26, 14, 21, 24, 13, 15, 18, 11, 28, 9, 37, 30, 10, 8, 11, 3, 1, 6, 10, 20]
Coordonnées des nœuds : [(40, 40), (22, 22), (36, 26), (21, 45), (45, 35), (55, 20), (33, 34), (50, 50), (55, 45), (26, 59), (40, 66), (55, 65), (35, 51), (62, 35), (62, 57), (62, 24), (21, 36), (33, 44), (9, 56), (62, 48), (66, 14), (44, 13), (26, 13), (11, 28), (7, 43), (17, 64), (41, 46), (55, 34), (35, 16), (52, 26), (43, 26), (31, 76), (22, 53), (26, 29), (50, 40), (55, 50), (54, 10), (60, 15), (47, 66), (30, 60), (30, 50), (12, 17), (15, 14), (16, 19), (21, 48), (50, 30), (51, 42), (50, 15), (48, 21), (12, 38), (15, 56), (29, 39), (54, 38), (55, 57), (67, 41

In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

8
Routes:
Route 1: 0 -> 9 (29.0) -> 10 (26.0) -> 12 (16.0) -> 31 (25.0) -> 34 (3.0) -> 39 (16.0) -> 40 (33.0) -> 0
Route 2: 0 -> 11 (37.0) -> 20 (22.0) -> 23 (6.0) -> 25 (14.0) -> 55 (7.0) -> 56 (26.0) -> 66 (37.0) -> 70 (11.0) -> 0
Route 3: 0 -> 1 (18.0) -> 28 (29.0) -> 30 (22.0) -> 33 (27.0) -> 38 (24.0) -> 48 (20.0) -> 61 (15.0) -> 65 (9.0) -> 73 (6.0) -> 0
Route 4: 0 -> 7 (15.0) -> 17 (20.0) -> 27 (17.0) -> 34 (16.0) -> 35 (10.0) -> 46 (27.0) -> 51 (12.0) -> 52 (19.0) -> 53 (22.0) -> 58 (21.0) -> 72 (1.0) -> 0
Route 5: 0 -> 2 (26.0) -> 4 (30.0) -> 8 (16.0) -> 19 (15.0) -> 29 (7.0) -> 41 (15.0) -> 43 (18.0) -> 45 (21.0) -> 74 (10.0) -> 75 (20.0) -> 0
Route 6: 0 -> 14 (31.0) -> 18 (13.0) -> 22 (12.0) -> 32 (28.0) -> 44 (4.0) -> 50 (22.0) -> 59 (24.0) -> 60 (13.0) -> 62 (18.0) -> 71 (3.0) -> 0
Route 7: 0 -> 6 (19.0) -> 16 (19.0) -> 21 (28.0) -> 26 (18.0) -> 29 (6.0) -> 36 (12.0) -> 47 (19.0) -> 63 (11.0) -> 67 (30.0) -> 68 (10.0) -> 69 (8.0) -> 0
Route 8: 0 -> 3 (11.0) -> 5 (21.0) -> 

Cas32

In [ ]:
from google.colab import files

# Fonction pour téléverser et charger un fichier d'entrée
def load_instance():
    print("Veuillez téléverser votre fichier d'entrée  :")
    uploaded = files.upload()  # Cette commande ouvre une fenêtre pour téléverser un fichier

    if not uploaded:
        print("Aucun fichier n'a été téléversé. Réessayez.")
        return None

    filename = list(uploaded.keys())[0]  # Récupère le nom du fichier téléversé

    # Lecture des données du fichier
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            n, Q = map(int, lines[0].split())  # Nombre de clients et capacité maximale des véhicules
            demands = list(map(int, lines[1].split()))  # Demandes des clients
            coords = [tuple(map(int, line.split())) for line in lines[2:]]  # Coordonnées des nœuds
        print("Fichier téléversé avec succès.")
        return n, Q, demands, coords
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {e}")
        return None

# Tester la fonction
result = load_instance()
if result:
    n, Q, demands, coords = result
    print("Nombre de clients :", n)
    print("Capacité des véhicules :", Q)
    print("Demandes des clients :", demands)
    print("Coordonnées des nœuds :", coords)
else:
    print("Impossible de charger les données.")

Veuillez téléverser votre fichier d'entrée  :


Saving Case32.txt to Case32.txt
Fichier téléversé avec succès.
Nombre de clients : 75
Capacité des véhicules : 220
Demandes des clients : [18, 26, 11, 30, 21, 19, 15, 16, 29, 26, 37, 16, 12, 31, 8, 19, 20, 13, 15, 22, 28, 12, 6, 27, 14, 18, 17, 29, 13, 22, 25, 28, 27, 19, 10, 12, 14, 24, 16, 33, 15, 11, 18, 17, 21, 27, 19, 20, 5, 22, 12, 19, 22, 16, 7, 26, 14, 21, 24, 13, 15, 18, 11, 28, 9, 37, 30, 10, 8, 11, 3, 1, 6, 10, 20]
Coordonnées des nœuds : [(40, 40), (22, 22), (36, 26), (21, 45), (45, 35), (55, 20), (33, 34), (50, 50), (55, 45), (26, 59), (40, 66), (55, 65), (35, 51), (62, 35), (62, 57), (62, 24), (21, 36), (33, 44), (9, 56), (62, 48), (66, 14), (44, 13), (26, 13), (11, 28), (7, 43), (17, 64), (41, 46), (55, 34), (35, 16), (52, 26), (43, 26), (31, 76), (22, 53), (26, 29), (50, 40), (55, 50), (54, 10), (60, 15), (47, 66), (30, 60), (30, 50), (12, 17), (15, 14), (16, 19), (21, 48), (50, 30), (51, 42), (50, 15), (48, 21), (12, 38), (15, 56), (29, 39), (54, 38), (55, 57), (67, 41

In [ ]:

import pulp
import math

# Calcul des distances euclidiennes
def euclidean_distance(i, j):
    dx = coords[j][0] - coords[i][0]
    dy = coords[j][1] - coords[i][1]
    return int(math.sqrt(dx**2 + dy**2) + 0.5)

# Déterminer le nombre maximal de véhicules nécessaires
max_vehicles = math.ceil(sum(demands) / Q)
print(max_vehicles)

# Créer le modèle
model = pulp.LpProblem("SD-VRP", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(n + 1) for j in range(n + 1) for k in range(1, max_vehicles + 1)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(1, n + 1) for k in range(1, max_vehicles + 1)), lowBound=0)

# Fonction objectif
model += pulp.lpSum(euclidean_distance(i, j) * x[i, j, k]
                    for i in range(n + 1) for j in range(n + 1) if i != j
                    for k in range(1, max_vehicles + 1))

# Contraintes
for i in range(1, n + 1):
    model += pulp.lpSum(y[i, k] for k in range(1, max_vehicles + 1)) == demands[i - 1]

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(y[i, k] for i in range(1, n + 1)) <= Q

for k in range(1, max_vehicles + 1):
    for i in range(n + 1):
        model += pulp.lpSum(x[i, j, k] for j in range(n + 1) if i != j) == pulp.lpSum(x[j, i, k] for j in range(n + 1) if i != j)

for k in range(1, max_vehicles + 1):
    model += pulp.lpSum(x[0, j, k] for j in range(1, n + 1)) <= 1

for i in range(1, n + 1):
    for k in range(1, max_vehicles + 1):
        model += y[i, k] <= demands[i - 1] * pulp.lpSum(x[j, i, k] for j in range(n + 1) if j != i)

# Résoudre avec une limite de temps augmentée et une configuration optimisée
status = model.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=30, threads=4))


# Afficher la solution
if pulp.LpStatus[status] == 'Optimal':
    routes = []
    truck_loads = []

    # Extraire les routes et les charges des camions
    for k in range(1, max_vehicles + 1):
        route = [0]
        load = 0
        delivered = {i: 0 for i in range(1, n + 1)}  # Livraisons effectuées par ce véhicule
        for i in range(1, n + 1):
            if pulp.value(y[i, k]) > 0:
                delivered[i] = pulp.value(y[i, k])
                route.append(i)
                load += delivered[i]
        route.append(0)
        if len(route) > 2:  # Ne pas inclure les véhicules inutilisés
            routes.append((route, delivered, load))
            truck_loads.append(load)

    # Recalcul du coût total à partir des routes
    recalculated_cost = 0
    for route, _, _ in routes:
        for i in range(len(route) - 1):
            recalculated_cost += euclidean_distance(route[i], route[i + 1])

    # Affichage des résultats
    print("Routes:")
    for k, (route, delivered, load) in enumerate(routes, start=1):
        route_str = " -> ".join(
            f"{node} ({round(delivered[node], 2)})" if node > 0 else str(node) for node in route
        )
        print(f"Route {k}: {route_str}")

    print(f"Total cost (recalculated): {recalculated_cost}")
    print(f"Number of deliveries: {sum(len(route) - 2 for route, _, _ in routes)}")
    print(f"Truck loads: {' '.join(map(str, truck_loads))}")
else:
    print(f"No optimal solution found. Solver status: {pulp.LpStatus[status]}")

7
Routes:
Route 1: 0 -> 6 (19.0) -> 8 (16.0) -> 16 (19.0) -> 17 (20.0) -> 19 (15.0) -> 27 (17.0) -> 29 (13.0) -> 45 (21.0) -> 51 (12.0) -> 52 (19.0) -> 63 (11.0) -> 68 (10.0) -> 0
Route 2: 0 -> 4 (30.0) -> 5 (21.0) -> 14 (31.0) -> 22 (12.0) -> 34 (19.0) -> 46 (27.0) -> 48 (20.0) -> 59 (24.0) -> 61 (15.0) -> 75 (20.0) -> 0
Route 3: 0 -> 3 (11.0) -> 7 (15.0) -> 12 (16.0) -> 21 (28.0) -> 32 (28.0) -> 35 (10.0) -> 40 (33.0) -> 44 (17.0) -> 47 (19.0) -> 53 (22.0) -> 0
Route 4: 0 -> 11 (37.0) -> 15 (8.0) -> 26 (18.0) -> 41 (15.0) -> 43 (18.0) -> 57 (14.0) -> 66 (37.0) -> 67 (30.0) -> 0
Route 5: 0 -> 10 (26.0) -> 23 (6.0) -> 25 (14.0) -> 28 (29.0) -> 31 (25.0) -> 42 (11.0) -> 55 (7.0) -> 56 (26.0) -> 62 (18.0) -> 64 (28.0) -> 0
Route 6: 0 -> 1 (18.0) -> 2 (26.0) -> 13 (12.0) -> 18 (13.0) -> 20 (22.0) -> 30 (22.0) -> 33 (27.0) -> 37 (14.0) -> 50 (22.0) -> 54 (16.0) -> 73 (6.0) -> 74 (10.0) -> 0
Route 7: 0 -> 9 (29.0) -> 24 (27.0) -> 36 (12.0) -> 38 (24.0) -> 39 (16.0) -> 49 (5.0) -> 58 (21.0) 